# اضافه کردن کتابخانه های مورد نیاز

In [95]:
from pandas_datareader import data as web
import pandas as pd 
import numpy as np
import datetime
import matplotlib.pyplot as plt
from pandas_datareader import data
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
from finta import TA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# گرفتن اطلاعات قیمتی ارز دیجیتال اتریوم 

In [96]:
data= pd.DataFrame(web.get_data_yahoo('USDEUR=X',start="20180621",end="20210420" ))
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-06-20,0.868500,0.859970,0.86378,0.86363,0.0,0.86363
2018-06-21,0.861940,0.856458,0.86155,0.86164,0.0,0.86164
2018-06-24,0.859850,0.854700,0.85759,0.85753,0.0,0.85753
2018-06-25,0.858220,0.853200,0.85457,0.85450,0.0,0.85450
2018-06-26,0.863290,0.856760,0.85851,0.85835,0.0,0.85835
...,...,...,...,...,...,...
2021-04-14,0.836270,0.833810,0.83456,0.83430,0.0,0.83430
2021-04-15,0.836757,0.833700,0.83537,0.83530,0.0,0.83530
2021-04-18,0.837280,0.830000,0.83554,0.83546,0.0,0.83546


In [97]:
crypto_data=pd.DataFrame()
crypto_data["Open"]=data.Open[:]
crypto_data["High"]=data.High[:]
crypto_data["Low"]=data.Low[:]
crypto_data["Close"]=data.Close[:]

# تقسیم داده ها به تست و آموزش 20 درصدی

In [98]:
sma9=TA.SMA(crypto_data, 9) # period : 9
sma9=sma9.reset_index(drop=True)
ind=pd.DataFrame(sma9)
ind.columns=['sma9']

In [99]:
ema9=TA.EMA(crypto_data,9) #period = 9
ema9=ema9.reset_index(drop=True)
ind['ema9']=ema9

In [100]:
y=crypto_data["Close"][8:]
y=y.reset_index(drop=True)
y=np.array(y)
x=ind.dropna()
x=np.array(x)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

# استفاده از روش درخت تصمیم

In [101]:
knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [103]:
test_preds = knn_model.predict(X_test)

In [73]:
clf = DecisionTreeRegressor()

In [74]:
clf.fit(X_train, y_train)

DecisionTreeRegressor()

In [104]:
clfP = clf.predict(X_test)

In [76]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [105]:
lrP = lr.predict(X_test)

In [78]:
crypto_data['returns'] = np.log(crypto_data['Close']/crypto_data['Close'].shift(1))

In [79]:
weights = np.ones((520,))
# Normalize
weights = weights / np.sum(weights)

In [80]:
def value_at_risk(value_invested, returns, weights, alpha=0.95, lookback_days=520):
    returns = returns.fillna(0.0)
    # Multiply asset returns by weights to get one weighted portfolio return
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    # Compute the correct percentile loss and multiply by value invested
    return np.percentile(portfolio_returns, 100 * (1-alpha)) * value_invested

In [81]:
value_invested = 1000000

value_at_risk(value_invested, crypto_data['returns'], weights, alpha=0.95)

-118.08113128540901

In [82]:
def cvar(value_invested, returns, weights, alpha=0.95, lookback_days=520):
    # Call out to our existing function
    var = value_at_risk(value_invested, returns, weights, alpha, lookback_days=lookback_days)
    returns = returns.fillna(0.0)
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    
    # Get back to a return rather than an absolute loss
    var_pct_loss = var / value_invested
    
    return value_invested * np.nanmean(portfolio_returns[portfolio_returns < var_pct_loss])

In [83]:
weights = np.ones((520,1))
# Normalize
weights = weights / np.sum(weights)

In [84]:
cvar(value_invested, crypto_data['returns'],weights)

<ipython-input-82-39f7c641cbdf>:10: RuntimeWarning: Mean of empty slice
  return value_invested * np.nanmean(portfolio_returns[portfolio_returns < var_pct_loss])


nan

## حد ضرر ثابت

In [106]:
vol=100
w=crypto_data["Close"]
z=ind.dropna().reset_index(drop=True)
z=z.reset_index(drop=True)
r,c=z.shape
f=0
sl=0
for i in range(1,r):
    if (f==-1):
        if(w[i]>sl):
            q=s/w[i]
            vol=vol*q
            f=0
    elif(f==1):
        if(w[i]<sl):
            q=w[i]/s
            vol=vol*q
            f=0
    if float(z['sma9'][i-1])<float(z['ema9'][i-1]):
        if float(z['sma9'][i])>float(z['ema9'][i]):
            if f==0:
                s=w[i]
                sl=w[i]*1.005
                f=-1
            elif f==1:
                q=w[i]/s
                vol=vol*q
                s=w[i]
                sl=w[i]*1.005
                f=-1
            else:
                continue
    if float(z['sma9'][i-1])>float(z['ema9'][i-1]):
        if float(z['sma9'][i])<float(z['ema9'][i]):
            if f==0:
                s=w[i]
                sl=w[i]*0.995
                f=1
            elif f==-1:
                q=s/w[i]
                vol=vol*q
                s=w[i]
                sl=w[i]*0.995
                f=1
            else:
                continue
print(vol)


62.14863521812405


## حد ضرر متحرک

In [107]:
vol=100
w=crypto_data["Close"]
z=ind.dropna().reset_index(drop=True)
z=z.reset_index(drop=True)
r,c=z.shape
f=0
sl=0
for i in range(2,r):
    if (f==-1):
        if(w[i]>sl):
            q=s/w[i]
            vol=vol*q
            f=0
        elif(w[i]<w[i-1]):
            sl=w[i]*1.05
    elif(f==1):
        if(w[i]<sl):
            q=w[i]/s
            vol=vol*q
            f=0
        elif(w[i]>w[i-1]):
            sl=w[i]*0.95
    if float(z['sma9'][i-1])<float(z['ema9'][i-1]):
        if float(z['sma9'][i])>float(z['ema9'][i]):
            if f==0:
                s=w[i]
                sl=w[i]*1.05
                f=-1
            elif f==1:
                q=w[i]/s
                vol=vol*q
                s=w[i]
                sl=w[i]*1.05
                f=-1
            else:
                continue
    if float(z['sma9'][i-1])>float(z['ema9'][i-1]):
        if float(z['sma9'][i])<float(z['ema9'][i]):
            if f==0:
                s=w[i]
                sl=w[i]*0.95
                f=1
            elif f==-1:
                q=s/w[i]
                vol=vol*q
                s=w[i]
                sl=w[i]*0.95
                f=1
            else:
                continue
print(vol)


73.76699739529914


مشاهده میشود در حالت حد ضرر متحرک سود بیشتری بدست آمده است